In [7]:
import pandas as pd
import numpy as np
import yfinance as yf

# ========== Input ==========
excel_file = "3.1 etf_with_ark_analysis_clean.xlsx"
sheet_name = "Comparable ETFs"
def get_risk_free_rate():
    irx = yf.Ticker("^IRX")
    rate = irx.history(period="5d")["Close"].dropna()
    if not rate.empty:
        return rate.iloc[-1] / 100 / 12
    else:
        raise ValueError("Unable to get risk free rate")

risk_free_rate = get_risk_free_rate()

# ========== Calculate ==========
def compute_max_drawdown(series):
    cummax = series.cummax()
    drawdown = (series - cummax) / cummax
    return drawdown.min()

def compute_return_to_mdd(returns):
    cumulative = (1 + returns).cumprod()
    mdd = compute_max_drawdown(cumulative)
    total_ret = cumulative.iloc[-1] - 1
    return total_ret / abs(mdd) if mdd != 0 else np.nan

def build_df(series_dict):
    df = pd.concat(series_dict, axis=1)
    df.columns = df.columns.get_level_values(0)  
    df.index.name = "Date"
    return df


# ========== Read ETF list ==========
df_etfs = pd.read_excel(excel_file, sheet_name=sheet_name, engine="openpyxl")
tickers = df_etfs.iloc[:, 0].dropna().astype(str).tolist()
print(f"✅ Get {len(tickers)} ETFs from Excel file.")

# ========== Initial ==========
daily_close_all = {}
sharpe_all = {}
rmd_all = {12: {}, 36: {}, 60: {}}
failed = []

# ========== Main ==========
for ticker in tickers:
    print(f"Handling: {ticker}")
    try:
        data = yf.download(ticker, start="2000-01-01", interval="1d", progress=False)
        close = data["Close"].dropna()
        if close.empty:
            print(f"No data for: {ticker}")
            failed.append(ticker)
            continue

        # Daily close price
        daily_close_all[ticker] = close

        # Monthly return
        monthly_close = close.resample("ME").last()
        returns = monthly_close.pct_change().dropna()

        # Annual Sharpe Ratio
        sharpe = returns.resample("YE").apply(
            lambda r: (r.mean() - risk_free_rate) / r.std() if r.std() > 0 else np.nan
        )
        sharpe_all[ticker] = sharpe

        # Return / Max Drawdown (1Y, 3Y, 5Y)
        for window in [12, 36, 60]:
            if len(returns) >= window:
                rmd = returns.rolling(window).apply(compute_return_to_mdd)
                rmd_all[window][ticker] = rmd
            else:
                rmd_all[window][ticker] = pd.Series(dtype=float)

        print(f"Finished: {ticker}{len(close)} ")

    except Exception as e:
        print(f"Error {ticker}: {e}")
        failed.append(ticker)

# ========== DataFrames ==========
df_close = build_df(daily_close_all)
df_sharpe = build_df(sharpe_all)
df_rmd_1y = build_df(rmd_all[12])
df_rmd_3y = build_df(rmd_all[36])
df_rmd_5y = build_df(rmd_all[60])

# ========== Write Excel ==========
print("\n📤 Writing Excel...")

df_close.reset_index().to_excel("4.1 daily_close.xlsx", index=False)
print("daily_close.xlsx Saved")

df_sharpe.reset_index().to_excel("4.2 sharpe_ratio.xlsx", index=False)
print("sharpe_ratio.xlsx Saved")

with pd.ExcelWriter("4.3 return_mdd.xlsx", engine="openpyxl") as writer:
    df_rmd_1y.reset_index().to_excel(writer, sheet_name="Return-MDD (1Y)", index=False)
    df_rmd_3y.reset_index().to_excel(writer, sheet_name="Return-MDD (3Y)", index=False)
    df_rmd_5y.reset_index().to_excel(writer, sheet_name="Return-MDD (5Y)", index=False)
print("return_mdd.xlsx Saved")

# report errors.
if failed:
    print(f"\nThere are {len(failed)} Ticker error:")
    print(", ".join(failed))
else:
    print("\nAll ETFs are successfully processed! ")


✅ Get 131 ETFs from Excel file.
Handling: ARKK
Finished: ARKK2629 
Handling: ARKG
Finished: ARKG2629 
Handling: ARKW
Finished: ARKW2652 
Handling: ARKF
Finished: ARKF1559 
Handling: ARKQ
Finished: ARKQ2652 
Handling: ARKX
Finished: ARKX1017 
Handling: ARKB
Finished: ARKB316 
Handling: IZRL
Finished: IZRL1850 
Handling: PRNT
Finished: PRNT2198 
Handling: ILDR
Finished: ILDR977 
Handling: BUYZ
Finished: BUYZ1291 
Handling: FDIF
Finished: FDIF458 
Handling: EVMT
Finished: EVMT746 
Handling: ARKZ
Finished: ARKZ355 
Handling: DYNI
Finished: DYNI355 
Handling: ARKW
Finished: ARKW2652 
Handling: SYNB
Finished: SYNB636 
Handling: TSLT
Finished: TSLT373 
Handling: WGMI
Finished: WGMI799 
Handling: AIYY
Finished: AIYY346 
Handling: SARK
Finished: SARK861 
Handling: TIME
Finished: TIME806 
Handling: MAGS
Finished: MAGS506 
Handling: CEPI
Finished: CEPI90 
Handling: FDG
Finished: FDG1266 
Handling: BMED
Finished: BMED1139 
Handling: MEDX
Finished: MEDX554 
Handling: DRAI
Finished: DRAI191 
Handlin

In [8]:
import pandas as pd
import numpy as np

# ========================== Load Data ==========================
df_close = pd.read_excel("4.1 daily_close.xlsx", index_col=0, parse_dates=True)
df_close.index = pd.to_datetime(df_close.index)
df_close = df_close.sort_index()
df_close = df_close.dropna(axis=1, how="all")

if "ARKK" not in df_close.columns:
    raise ValueError("ARKK is not in the daily_close data.")

etfs = [col for col in df_close.columns if col != "ARKK"]
start_date = df_close["ARKK"].dropna().index.min()
df_close = df_close[df_close.index >= start_date]

# ========================== Return Calculation ==========================
daily_ret = df_close.pct_change(fill_method=None)
weekly_ret = df_close.resample("W-FRI").last().pct_change(fill_method=None)
monthly_ret = df_close.resample("M").last().pct_change(fill_method=None)

# ========================== Rolling Correlation Function ==========================
def compute_fixed_window_rolling_corr(df, target_col, etf_list, window, label=""):

    result = {}
    for etf in etf_list:
        if etf == target_col or etf not in df.columns:
            continue

        aligned = df[[target_col, etf]].dropna()
        if len(aligned) < window:
            print(f"⚠️ Skipping {etf} ({label}): not enough data ({len(aligned)})")
            continue

        def corr_fn(x):
            x = x.reshape(-1, 2)
            if np.std(x[:, 0]) == 0 or np.std(x[:, 1]) == 0:
                return np.nan
            return np.corrcoef(x[:, 0], x[:, 1])[0, 1]

        rolling_corr = aligned.rolling(window).apply(corr_fn, raw=True)
        result[etf] = rolling_corr[target_col]  

    df_result = pd.DataFrame(result)
    df_result.index.name = "Date"
    return df_result



# ========================== Rolling Correlation Calculation ==========================
etfs = [col for col in daily_ret.columns if col != "ARKK"]

rolling_corr_daily = compute_fixed_window_rolling_corr(daily_ret, "ARKK", etfs, window=60, label="daily")
rolling_corr_weekly = compute_fixed_window_rolling_corr(weekly_ret, "ARKK", etfs, window=26, label="weekly")
rolling_corr_monthly = compute_fixed_window_rolling_corr(monthly_ret, "ARKK", etfs, window=12, label="monthly")


# ========================== Save to Excel ==========================
print("💾 Saving to 'rolling_correlation_with_ARKK.xlsx'...")
with pd.ExcelWriter("4.4 rolling_correlation_with_ARKK.xlsx", engine="openpyxl") as writer:
    rolling_corr_daily.to_excel(writer, sheet_name="Daily")
    rolling_corr_weekly.to_excel(writer, sheet_name="Weekly")
    rolling_corr_monthly.to_excel(writer, sheet_name="Monthly")

print("✅ Done. File saved as 'rolling_correlation_with_ARKK.xlsx'.")


/var/folders/7g/kq56s8d958b3kd1gdqwgbtzc0000gr/T/ipykernel_55188/2280100599.py:20: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_ret = df_close.resample("M").last().pct_change(fill_method=None)


⚠️ Skipping BEGS (daily): not enough data (45)
⚠️ Skipping JXX (daily): not enough data (48)
⚠️ Skipping MVLL (daily): not enough data (27)
⚠️ Skipping MRAL (daily): not enough data (27)
⚠️ Skipping COIW (daily): not enough data (39)
⚠️ Skipping CEPI (weekly): not enough data (19)
⚠️ Skipping TEK (weekly): not enough data (25)
⚠️ Skipping WAR (weekly): not enough data (15)
⚠️ Skipping BAI (weekly): not enough data (25)
⚠️ Skipping GROZ (weekly): not enough data (19)
⚠️ Skipping LFSC (weekly): not enough data (24)
⚠️ Skipping VOLT (weekly): not enough data (19)
⚠️ Skipping BEGS (weekly): not enough data (10)
⚠️ Skipping TTEQ (weekly): not enough data (25)
⚠️ Skipping JXX (weekly): not enough data (10)
⚠️ Skipping MVLL (weekly): not enough data (6)
⚠️ Skipping MRAL (weekly): not enough data (6)
⚠️ Skipping COIW (weekly): not enough data (8)
⚠️ Skipping CEPI (monthly): not enough data (4)
⚠️ Skipping DRAI (monthly): not enough data (9)
⚠️ Skipping TEK (monthly): not enough data (6)
⚠️ Ski

In [9]:
top_etfs = rolling_corr_daily.mean().sort_values(ascending=False).head(15).index.tolist()
print("Top correlated ETFs with ARKK:\n", top_etfs)


Top correlated ETFs with ARKK:
 ['PTIR', 'TEKX', 'MSTU', 'LIVR', 'DECO', 'BKIV', 'CHAT', 'FDFF', 'FDIF', 'FDCF', 'FMED', 'FBOT', 'FDTX', 'MAGS', 'FWD']


In [18]:
# ========================== Pearson Correlation with ARKK ==========================
print("📊 Calculating full-period Pearson correlation with ARKK...")

# Align ARKK with all other ETFs and drop rows with missing values
aligned_ret = daily_ret[["ARKK"] + [col for col in daily_ret.columns if col != "ARKK"]].dropna()

# Compute Pearson correlation between ARKK and each ETF
# .corr() by default computes Pearson correlation
pearson_corr = aligned_ret.corr()["ARKK"].drop("ARKK").sort_values(ascending=False)

# Print the top 15 ETFs most correlated with ARKK
print("Top 15 ETFs by Pearson correlation with ARKK:")
print(pearson_corr.head(15))

# Save the full correlation series to Excel
pearson_corr.to_frame(name="Pearson Correlation with ARKK").to_excel("4.5 pearson_corr_with_ARKK.xlsx")
print("✅ Saved as '4.5 pearson_corr_with_ARKK.xlsx'")


📊 Calculating full-period Pearson correlation with ARKK...
Top 15 ETFs by Pearson correlation with ARKK:
TARK    0.999503
OARK    0.991209
ARKW    0.988173
ARKF    0.985904
FDG     0.980586
PJFG    0.976013
GROZ    0.975097
AGIX    0.973985
KQQQ    0.972622
IETC    0.972472
JTEK    0.971392
FDND    0.970473
AOTG    0.969319
TTEQ    0.969129
AMOM    0.968918
Name: ARKK, dtype: float64
✅ Saved as '4.5 pearson_corr_with_ARKK.xlsx'
